# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [7]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [8]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [9]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [10]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
len(X_train)

1051

## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [24]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(Dense(50, activation='relu'))

# Output layer
baseline_model.add(Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [25]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [26]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [27]:
# Train the model 
normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [28]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [29]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, y_train_scaled, epochs=150, batch_size=32, validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: 0.5575 - mse: 0.5575 - val_loss: 0.2199 - val_mse: 0.2199
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2655 - mse: 0.2655 - val_loss: 0.2021 - val_mse: 0.2021
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2206 - mse: 0.2206 - val_loss: 0.1559 - val_mse: 0.1559
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1832 - mse: 0.1832 - val_loss: 0.1452 - val_mse: 0.1452
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1563 - mse: 0.1563 - val_loss: 0.1628 - val_mse: 0.1628
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1401 - mse: 0.1401 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1250 - mse: 0.1250 - val_loss: 0.1332 - val_mse: 0.1332
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1146 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1241 - val_mse: 0.1241
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0220 - mse: 0.0220 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1231 - val_mse: 0.1231
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1207 - val_mse: 0.1207
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1207 - val_mse: 0.1207
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0202 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1263 - val_mse: 0.1263
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1274 - val_mse: 0.1274
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1294 - val_mse: 0.1294
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1295 - val_mse: 0.1295
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 881us/step - loss: 0.0069 - mse: 0.0069


[0.006868824362754822, 0.006868824362754822]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [31]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1303 - mse: 0.1303


[0.1302611231803894, 0.1302611231803894]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [33]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28362.133268435515

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [34]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.4827 - mse: 0.4827 - val_loss: 0.2020 - val_mse: 0.2020
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2377 - mse: 0.2377 - val_loss: 0.1721 - val_mse: 0.1721
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1995 - mse: 0.1995 - val_loss: 0.1675 - val_mse: 0.1675
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1659 - mse: 0.1659 - val_loss: 0.1717 - val_mse: 0.1717
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1511 - mse: 0.1511 - val_loss: 0.1757 - val_mse: 0.1757
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1396 - mse: 0.1396 - val_loss: 0.1658 - val_mse: 0.1658
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1255 - mse: 0.1255 - val_loss: 0.1752 - val_mse: 0.1752
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1157 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.2088 - val_mse: 0.2088
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0246 - mse: 0.0246 - val_loss: 0.2110 - val_mse: 0.2110
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.2082 - val_mse: 0.2082
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.2094 - val_mse: 0.2094
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.2103 - val_mse: 0.2103
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.2098 - val_mse: 0.2098
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.2101 - val_mse: 0.2101
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0217 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0117 - mse: 0.0117 - val_loss: 0.2211 - val_mse: 0.2211
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.2186 - val_mse: 0.2186
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.2208 - val_mse: 0.2208
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.2184 - val_mse: 0.2184
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.2202 - val_mse: 0.2202
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.2193 - val_mse: 0.2193
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.2223 - val_mse: 0.2223
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0094 - mse: 0.0094


[0.009382984600961208, 0.009382984600961208]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [36]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.2249 - mse: 0.2249


[0.22490978240966797, 0.22490978240966797]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [37]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3725 - mse: 0.3725 - val_loss: 0.1348 - val_mse: 0.1348
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2142 - mse: 0.2142 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1724 - mse: 0.1724 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1532 - mse: 0.1532 - val_loss: 0.1046 - val_mse: 0.1046
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1350 - mse: 0.1350 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1257 - mse: 0.1257 - val_loss: 0.1067 - val_mse: 0.1067
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1152 - mse: 0.1152 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1073 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1144 - val_mse: 0.1144
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0239 - mse: 0.0239 - val_loss: 0.1128 - val_mse: 0.1128
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1153 - val_mse: 0.1153
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0225 - mse: 0.0225 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0223 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1213 - val_mse: 0.1213
Epoch 127/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1219 - val_mse: 0.1219
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1237 - val_mse: 0.1237
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [38]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0080 - mse: 0.0080


[0.00798596628010273, 0.00798596628010273]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [39]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.1218 - mse: 0.1218


[0.1218261867761612, 0.1218261867761612]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [46]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 10ms/step - loss: 0.3112 - mse: 0.3112 - val_loss: 0.1251 - val_mse: 0.1251
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1858 - mse: 0.1858 - val_loss: 0.1179 - val_mse: 0.1179
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1421 - mse: 0.1421 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1087 - mse: 0.1087 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0889 - mse: 0.0889 - val_loss: 0.1010 - val_mse: 0.1010
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0802 - mse: 0.0802 - val_loss: 0.1134 - val_mse: 0.1134
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0616 - mse: 0.0616 - val_loss: 0.1473 - val_mse: 0.1473
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0568 - 

33/33 [==============================] - 0s 2ms/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1077 - val_mse: 0.1077
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1162 - val_mse: 0.1162
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0077 - mse: 0.0077 - val_loss: 0.1230 - val_mse: 0.1230
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.0961 - val_mse: 0.0961
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0066 - mse: 0.0066 - val_loss: 0.1196 - val_mse: 0.1196
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1159 - val_mse: 0.1159
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0075 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0064 - mse: 0.0064 - val_loss: 0.0858 - val_mse: 0.0858
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0886 - val_mse: 0.0886
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0891 - val_mse: 0.0891
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0060 - mse: 0.0060 - val_loss: 0.0906 - val_mse: 0.0906
Epoch 129/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0826 - val_mse: 0.0826
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.0875 - val_mse: 0.0875
Epoch 131/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0854 - val_mse: 0.0854
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [47]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0043 - mse: 0.0043


[0.004320498555898666, 0.004320498555898666]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [48]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0950 - mse: 0.0950


[0.09497465938329697, 0.09497465938329697]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [49]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', loss='mse', metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 6ms/step - loss: 0.3475 - mse: 0.3475 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1534 - mse: 0.1534 - val_loss: 0.1074 - val_mse: 0.1074
Epoch 3/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1059 - mse: 0.1059 - val_loss: 0.1247 - val_mse: 0.1247
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0817 - mse: 0.0817 - val_loss: 0.1067 - val_mse: 0.1067
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0668 - mse: 0.0668 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0469 - mse: 0.0469 - val_loss: 0.1191 - val_mse: 0.1191
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0376 - mse: 0.0376 - val_loss: 0.1110 - val_mse: 0.1110
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0300 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0022 - mse: 0.0022 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0016 - mse: 0.0016 - val_loss: 0.1052 - val_mse: 0.1052
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0014 - mse: 0.0014 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1049 - val_mse: 0.1049
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0010 - mse: 0.0010 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0012 - mse: 0.0012 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 69/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 70/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0025 - mse: 0

33/33 [==============================] - 0s 2ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0954 - val_mse: 0.0954
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.1023 - val_mse: 0.1023
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0036 - mse: 0.0036 - val_loss: 0.0906 - val_mse: 0.0906
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0028 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [50]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 1ms/step - loss: 0.0024 - mse: 0.0024


[0.0024368641898036003, 0.0024368641898036003]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [51]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 1ms/step - loss: 0.0916 - mse: 0.0916


[0.09158008545637131, 0.09158008545637131]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [52]:
# Evaluate the best model on test data
adam_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 1ms/step - loss: 0.1539 - mse: 0.1539


[0.15387071669101715, 0.15387071669101715]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [53]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

30825.45310278544

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.